# Project Two
## Group 5: Brian, Josh, Jeff, Yuliya, Natalie
2016 Election and Police/Judicial Employment

In [108]:
# Import packages
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import os

# (E) EXTRACTION

### Load in police data

In [109]:
# Define directory to police files
police_dir = "Resources/ucr-police-employee-data-2016-by-state-by-counties/"

# Find paths to all police files
police_paths = os.listdir(police_dir)

# Initialize df list
police_df_list = []

# Iterate through each path
for path in police_paths:
    # Identify state of origin
    state = pd.read_excel(police_dir + path, header = 0).iloc[0,0]
    
    # Load in data
    police_df = pd.read_excel(police_dir + path, header = 4)

    # Add state column
    police_df['State'] = state
    
    # Remove '\n' from column names
    police_df = police_df.rename(columns={'Metropolitan/Nonmetropolitan': 'Metropolitan',
                                          'Total law\nenforcement\nemployees': 'Total Law Enforcement Employees',
                                          'Total\nofficers': 'Total Officers',
                                          'Total\ncivilians': 'Total Civilians'})
    
    # Append to list
    police_df_list.append(police_df)

# Merge dataframes together
police_df = pd.concat(police_df_list, axis = 0)
police_df.head()

,Metropolitan,County,Total Law Enforcement Employees,Total Officers,Total Civilians,State
0,Metropolitan Counties,Anoka,256.0,129.0,127.0,MINNESOTA
1,NaN,Benton,70.0,24.0,46.0,MINNESOTA
2,NaN,Blue Earth,69.0,31.0,38.0,MINNESOTA
3,NaN,Carlton,52.0,22.0,30.0,MINNESOTA
4,NaN,Carver,148.0,77.0,71.0,MINNESOTA


### Load in voting and county data

In [110]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///Resources/archive/database.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Create a session
session = Session(engine)

# Create a connection
conn = engine.connect()

# Generate dataframe
voting_df = pd.read_sql_query('select * from primary_results', con = engine)
county_df = pd.read_sql_query('select * from county_facts', con = engine)
county_dict_df = pd.read_sql_query('select * from county_facts_dictionary', con = engine)

# (T) TRANSFORM

### Format Police Data

In [111]:
# Modify values in Metropolitan column to be more readable
police_df = police_df.drop('Metropolitan', axis = 1)

# Modify State to be first capitalized only
police_df['State'] = police_df['State'].str.capitalize()
police_df.head()

,County,Total Law Enforcement Employees,Total Officers,Total Civilians,State
0,Anoka,256.0,129.0,127.0,Minnesota
1,Benton,70.0,24.0,46.0,Minnesota
2,Blue Earth,69.0,31.0,38.0,Minnesota
3,Carlton,52.0,22.0,30.0,Minnesota
4,Carver,148.0,77.0,71.0,Minnesota


### Format Voting Data

In [112]:
# Make column names consistent
voting_df = voting_df.rename(columns={'county': 'County',
                                      'state': 'State'})

voting_df.head()

,State,state_abbreviation,County,fips,party,candidate,votes,fraction_votes
0,Alabama,AL,Autauga,1001,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005,Democrat,Bernie Sanders,222,0.078


### Format County Data

In [114]:
# Remove state and country rows
county_df = county_df[ county_df['state_abbreviation'] != "" ]

# Subset shared states with voting_df for mapping state names
shared_states = list(set(county_df['state_abbreviation']) & set(voting_df['state_abbreviation']))
county_df = county_df[county_df['state_abbreviation'].isin(shared_states)]

# Add full state name column
state_dict = dict(zip(voting_df['state_abbreviation'],voting_df['State']))
county_df['State'] = county_df['state_abbreviation'].map(state_dict)
county_df.head()

# Subset for columns of interest
columns = ['PST045214', 'POP010210', 'POP060210', 'LND110210', 'PST120214', 
           'EDU635213', 'EDU685213', 'INC910213', 'INC110213']
county_dict_df = county_dict_df[county_dict_df['column_name'].isin(columns)]
county_df = county_df.loc[:,county_df.columns.isin(['area_name', 'fips', 'state_abbreviation', 'State',
                                                    columns[0], columns[1], columns[2], columns[3], columns[4],
                                                    columns[5], columns[6], columns[7], columns[8]])]
county_df.head()

# Make shorter column name descriptions
county_dict_df['short_description'] = ['2014_pop', 'pct_pop_delt10to14', '2010_pop', 'pct_post_hs_grad', 'pct_post_bach_grad',
                                       'per_capita_income', 'median_household_income', 'sq_miles', 'pop_per_sq_mile']

# Map column names of county_df to short description
column_name_dict = dict(zip(county_dict_df['column_name'], county_dict_df['short_description']))
county_df = county_df.rename(column_name_dict, axis = 1)

,column_name,description,short_description
0,PST045214,"Population, 2014 estimate",2014_pop
2,PST120214,"Population, percent change - April 1, 2010 to ...",pct_pop_delt10to14
3,POP010210,"Population, 2010",2010_pop
19,EDU635213,"High school graduate or higher, percent of per...",pct_post_hs_grad
20,EDU685213,"Bachelor's degree or higher, percent of person...",pct_post_bach_grad
29,INC910213,Per capita money income in past 12 months (201...,per_capita_income
30,INC110213,"Median household income, 2009-2013",median_household_income
49,LND110210,"Land area in square miles, 2010",sq_miles
50,POP060210,"Population per square mile, 2010",pop_per_sq_mile


['area_name', 'fips', 'state_abbreviation', 'State', ['PST045214', 'POP010210', 'POP060210', 'LND110210', 'PST120214', 'EDU635213', 'EDU685213', 'INC910213', 'INC110213']]
